<a href="https://colab.research.google.com/github/hecshzye/natural_language_processing-cases/blob/main/disaster_tweet_detection_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disaster Tweets Detection using Natural Language Processing

### The goal is to predict which tweets are about real disasters and which are not using `NLP` and `TensorFlow`

- The dataset used in this model is the `Real-or-Not` from `Kaggle competition`: https://www.kaggle.com/c/nlp-getting-started/data


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import os

In [3]:
# Importing few functions wriiten for workflow and ease
!wget https://raw.githubusercontent.com/hecshzye/natural_language_processing-cases/main/helper_functions.py

--2022-01-14 04:45:09--  https://raw.githubusercontent.com/hecshzye/natural_language_processing-cases/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6442 (6.3K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   6.29K  --.-KB/s    in 0s      

2022-01-14 04:45:10 (91.4 MB/s) - ‘helper_functions.py’ saved [6442/6442]



In [4]:
from helper_functions import plot_loss_curves, create_confusion_matrix, create_tensorboard_callback, unzip_data, compare_history